In [1]:
from time import sleep
from ngsolve.fem import CompilePythonModule
from pathlib import Path
from ngsolve import *
from netgen.geom2d import unit_square
from netgen import geom2d
from ngsolve.webgui import Draw

In [2]:
def UnitTrig():
    geo = geom2d.SplineGeometry()
    p1 = geo.AppendPoint (0,0)
    p2 = geo.AppendPoint (1,0)
    p3 = geo.AppendPoint (0,1)
    geo.Append (["line", p1, p2])
    geo.Append (["line", p2, p3])
    geo.Append (["line", p3, p1])
    return Mesh(geo.GenerateMesh(maxh=1))

def TrigA():
    geo = geom2d.SplineGeometry()
    p1 = geo.AppendPoint (0,0)
    p2 = geo.AppendPoint (1,0)
    p3 = geo.AppendPoint (1,1)
    geo.Append (["line", p1, p2])
    geo.Append (["line", p2, p3])
    geo.Append (["line", p3, p1])
    return Mesh(geo.GenerateMesh(maxh=1))
def TrigB():
    geo = geom2d.SplineGeometry()
    p1 = geo.AppendPoint (1,0)
    p2 = geo.AppendPoint (1,1)
    p3 = geo.AppendPoint (0,1)
    geo.Append (["line", p1, p2])
    geo.Append (["line", p2, p3])
    geo.Append (["line", p3, p1])
    return Mesh(geo.GenerateMesh(maxh=1))

def UnitSquare():
    geo = geom2d.SplineGeometry()
    p1 = geo.AppendPoint (-1,-1)
    p2 = geo.AppendPoint (1,-1)
    p3 = geo.AppendPoint (1,1)
    p4 = geo.AppendPoint (-1,1)
    geo.Append (["line", p1, p2])
    geo.Append (["line", p2, p3])
    geo.Append (["line", p3, p4])
    geo.Append (["line", p4, p1])
    return Mesh(geo.GenerateMesh (maxh=0.1))

def TwoTrigs(_maxh=1, antidiagonal=False):
    geo = geom2d.SplineGeometry()
    p1 = geo.AppendPoint (0,0)
    p2 = geo.AppendPoint (1,0)
    p3 = geo.AppendPoint (0,1)
    p4 = geo.AppendPoint (1,1)
    geo.Append (["line", p1, p2])
    if antidiagonal:
        geo.Append (["line", p2, p3], leftdomain=1,rightdomain=1)
    geo.Append (["line", p3, p1])
    geo.Append (["line", p2, p4])
    geo.Append (["line", p4, p3])
    return Mesh(geo.GenerateMesh(maxh=_maxh))

In [3]:
mm = CompilePythonModule(Path('mymodule.cpp'), init_function_name='mymodule')

NgException: problem calling compiler

In [ ]:
mesh=TwoTrigs(_maxh=2**-6)
#fes = mm.HDivLumpingFESpace(mesh, verbose=True)
fes = mm.HDivLumpingFESpace(mesh, verbose=False)
hdivfes = HDiv(mesh, order=1, RT=True)
gfu = GridFunction(fes)

In [ ]:
uex=(2*exp(-50*((x+1))**2),0)

gfu.Set(uex)
Draw(gfu, order=3, vectors={"grid_size" : 40})

In [ ]:
lumping=IntegrationRule( [(0,0),(1,0),(0,1),(1/3,1/3)], [3/8,3/8,3/8,1/24])

u,v=fes.TnT()
mform=u*v*dx#(intrules={TRIG : lumping})
aform=div(u)*div(v)*dx

m=BilinearForm(mform).Assemble()
a=BilinearForm(aform).Assemble()

In [ ]:
rows,cols,vals = m.mat.COO()
import scipy.sparse as sp
A = sp.csr_matrix((vals,(rows,cols)))
import matplotlib.pylab as plt
plt.spy(A,precision=1e-16)
plt.show()

In [ ]:
tau = 0.0005
tend = 1
u0 = (2*exp(-50*(x)**2),2*exp(-50*(x)**2))# (-sin(pi*(x))*cos(0*pi),0)
v0 = 0
t=1
uex= (cos(pi*x)*cos(t*pi),0)

gfu = GridFunction(fes)
gfu.Set(u0)

scene = Draw((gfu), mesh, order=2, autoscale=False, eval=0)
sleep (1)
unew = gfu.vec.CreateVector()
uold = gfu.vec.CreateVector()
uold.data = gfu.vec

def VertexPatchBlocks(mesh, fes):
    nedges=fes.GetNEdges()
    nel=fes.GetNE()
    blocks = []
    freedofs = fes.FreeDofs()
    for v in mesh.vertices:
        vdofs=set()
        for e in mesh[v].edges:
            dofs=fes.GetDofNrs(e)
            if (v in mesh[e].vertices):
                vdofs.add(dofs[mesh[e].vertices.index(v)])
#                print(v, ":",e,":",mesh[e].vertices,":",dofs)
#            i=0
#            for ed in mesh[el].edges:
#                blocks[mesh[ed].vertices[0].nr].add(dofs[i])
#                blocks[mesh[ed].vertices[1].nr].add(dofs[i + 4])
#                i+=1#
        blocks.append(vdofs)
    for e in mesh.Elements(VOL):
        a=set()
        a|=set([2*nedges+2*e.nr])
        a|=set([2*nedges+2*e.nr+1])
        blocks.append(a)
    return blocks

blocks = VertexPatchBlocks(mesh, fes)
blockjac = m.mat.CreateBlockSmoother(blocks)

minv=m.mat.Inverse(inverse='umfpack')

for n in range(int(tend/tau)):
    unew.data = 2*gfu.vec - uold 
    unew.data -= tau**2 * (minv*(a.mat * gfu.vec))
    #unew.data -= tau**2 * blockjac@a.mat * gfu.vec
    uold.data = gfu.vec
    gfu.vec.data = unew.data
    if n % 20 == 0:
        scene.Redraw()
scene.Redraw()



In [ ]:
for e in mesh.Elements(VOL):
    print(fes.GetDofNrs(e))
    print(hdivfes.GetDofNrs(e))

In [ ]:
for i in range(fes.ndof):#[1, 2, 0, 4, 5, 3]:
    gfu.vec[:]=0
    gfu.vec[i]=1
    Draw(gfu, order=3,vectors={"grid_size":40})